In [2]:
from utils import *
from tonotopy import *
import findpeaks
from skimage import measure
import os

t_pre = 0.5#0.2
t_post = 0.50#0.300
bin_width = 0.005
# Créer les bins de temps"
psth_bins = np.arange(-t_pre, t_post, bin_width)

In [5]:
path = "/auto/data2/eTheremin/ALTAI/ALTAI_20240807_SESSION_00/"


In [3]:
t_pre = 0.5#0.2
t_post = 0.50#0.300
bin_width = 0.005
# Créer les bins de temps"
psth_bins = np.arange(-t_pre, t_post, bin_width)

In [6]:
data = np.load(path+'headstage_0/data_0.005.npy', allow_pickle=True)
features = np.load(path+'headstage_0/features_0.005.npy', allow_pickle=True)
gc = np.load(path+'headstage_0/good_clusters.npy', allow_pickle=True)


In [7]:
tones = get_played_frequency(features, t_pre, t_post, bin_width, 'playback')
# prendre les valeurs uniques de tones
unique_tones = sorted(np.unique(tones))

In [ ]:
def plot_heatmap_bandwidth_both_conditions(heatmaps_tr, heatmaps_pb,threshold, gc,unique_tones, min_freq, max_freq, bin_width, psth_bins, t_pre,path, folder, condition):
    """""
    Best function pour déterminer la bandwidth et plotter la heatmap et les contours de la bandwidth
    input : heatmaps(contenant plusieurs clusters), le threshold pour la detection du pic, good_clusters
        unique_tones (les fréquences jouées), min_freq, max_freq : les indices des fréquences qu'on exclut (pas assez de présentations)
        condition : 'tracking' ou 'playback'
    output : save plot des heatmap avec la bandwidth entourée .png
            save tableau des heatmaps telles que plottée (avec les psth) .npy
            save tableau contenant les bandwidth de chaque cluster .npy
            
    """
    heatmaps = [heatmaps_tr, heatmaps_pb]

    vmax_tr = 2 * np.std(heatmaps_tr)
    vmax_pb = 2 * np.std(heatmaps_pb)

    vmax = np.max(vmax_pb, vmax_tr)
    vmin = -np.max(vmax_pb, vmax_tr)
    # pour les plots:
    for heatmap in heatmaps :
    #num_rows, num_columns = get_plot_geometry(gc)
    
        num_plots, num_rows, num_columns = get_better_plot_geometry(gc)


        # Create a figure with subplots
        fig, axes = plt.subplots(4, 8, figsize=(16, 8))
        fig.suptitle(f'Heatmaps clusters {condition}', y=1.02)
        plt.subplots_adjust() 
        
        # Flatten the axis array if it's more than 1D
        #if num_rows > 1 and num_columns > 1:
            #axes = axes.flatten()
        #
        bandwidth = []
        plotted_heatmap = []
        peaks = []
        for n, cluster in enumerate(gc):
            if cluster < num_plots:
                row, col = get_plot_coords(n)
                #print(cluster)
                heatmap_cluster = np.array(heatmaps[n])
                hm, peak = detect_peak(heatmaps, n)
                #heatmap_min = np.min(heatmap_cluster)
                #heatmap_max = np.max(abs(heatmap_cluster))
                #abs_max = max(abs(heatmap_min), abs(heatmap_max))
                #abs_max = np.max(abs(heatmap_cluster[min_freq:-max_freq]))
                abs_max = np.max(abs(heatmap_cluster[3:-3]))*0.4
                contours = get_contour(hm, threshold)
                #j'essaye en prenant la absolute value de hm
                #contours = get_contour(np.abs(hm), threshold)
                
            # Je retire la moyenne pre-stim ligne par ligne (fréquence par fréquence)
                t_0 = int(t_pre/bin_width)
                prestim_hm = heatmap_cluster[:, :t_0]
                mean_freq = np.mean(prestim_hm, axis=1)

                for i in range(heatmap_cluster.shape[0]):  # Parcours des lignes de A
                    heatmap_cluster[i] -= mean_freq[i]
                
                
                smoothed = smooth_2d(heatmap_cluster, 5)
                
                #je mets des zeros aux frequences trop hautes et trop basses où je n'ai pas
                #assez de présentations
                #lowf = np.zeros((min_freq+1, len(smoothed[0])))
                #highf = np.zeros((max_freq+1, len(smoothed[0])))
                
                #milieu = np.concatenate((lowf, smoothed[min_freq:-max_freq]))

                # Concaténation à l'arrière
                #milieu = np.concatenate((milieu, highf))

                vmin = -2 * np.std(smoothed)
                vmax = 2 * np.std(smoothed)

                norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
                img = axes[row, col].pcolormesh(smoothed, cmap='seismic', norm=norm)
                axes[row, col].set_xlabel('Time')
                #axes[row, col].set_yticks(np.arange(len(unique_tones)), unique_tones)
                axes[row, col].set_xlabel('Time')
                #axes[row, col].set_ylabel('Frequency [Hz]')
                axes[row, col].set_title(f'Cluster {cluster}')
                axes[row, col].axvline(x=t_0, color='black', linestyle='--') # to print a vertical line at the stim onset time
            
            

                #Je ne prends la réponse qu'entre 40 et 60ms
                #max = 0
                #min = len(unique_tones[min_freq:-max_freq])-2
                max_length =  0    
                x_c, y_c, minf, maxf = np.nan, np.nan, 0,0 # au cas où on trouve pas de contour
                for contour in contours:
                    if ((contour[:, 1] > t_0-5).all() and (contour[:, 1] < t_0+10).all()):
                        if len(contour[:, 0])>max_length:
                            max_length = len(contour[:, 0])
                            x_c = contour[:, 1]
                            y_c = contour[:, 0]
                            maxf = np.max(contour[:, 0])
                            minf = np.min(contour[:, 0])
                            test = contour[:, 0]
                            if maxf<len(unique_tones)-1:
                                maxf+=1
                #axes[row, col].plot(x_c, y_c, linewidth=2, color='green')
                #print(plotted_freq[int(min)], plotted_freq[int(max)])
                # je mets np.nan dans bandwidth si je ne trouve pas de contour
                if max_length==0 or maxf==0:
                    bandwidth.append([np.nan, np.nan])
                    peaks.append(np.nan)
                else : 
                
                #je prends +1 dans le maxf
                    bandwidth.append([unique_tones[int(minf)], unique_tones[int(maxf)]])
                    peaks.append(unique_tones[peak[0]])
                plotted_heatmap.append(smoothed)
                #cbar_ax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
                #fig.colorbar(img, cax=cbar_ax)
            # Hide any unused subplots
        for ax in axes[num_plots:]:
            ax.axis('off')
    plt.tight_layout()  
    plt.savefig(path+folder+f'/heatmap_{condition}.png') # save the figure of the heatmap
    np.save(path+folder+f'/heatmap_bandwidth.npy', bandwidth) # save the values of the bandwidth
    np.save(path+folder+f'/heatmap_plot_{condition}.npy',plotted_heatmap ) # save the values of the heatmap as it is plotted 
    np.save(path+folder+f'/best_frequency_{condition}.npy', peaks)
    return ('all izz well')

In [ ]:
if not os.path.exists(path + 'heatmap_plot_playback.npy'):
    print("calculating heatmaps")
    heatmaps = get_tonotopy(data, features, t_pre, t_post, bin_width, gc, unique_tones, 0, 0, 'playback', 'heatmaps')

else:
    heatmaps = np.load(path + 'heatmap_plot_playback.npy', allow_pickle = True)
    print('heatmaps already exist')

#récupérer les heatmaps
plot_heatmap_bandwidth(heatmaps,3, gc,unique_tones, 2, 2, bin_width, psth_bins, t_pre,path, '', 'playback')